In [ ]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [ ]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the Alpaca api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

# Data Collection

In this step, you will need to use the Alpaca api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [ ]:
# Get all Asstes
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])
# Display the first 10 asset tickers
# Set the ticker
display(asset_info_df.head(10))
myportfolio=["AGG","SPP"]
 
 
timeframe = '1D'
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)
# Get 1 year's worth of historical data for AGG,SPY

df=api.get_barset(
    myportfolio,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
    ).df
 
df.head(2)
 

In [ ]:
# Drop Outer Table Level
agg = agg.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
agg.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
agg.index = df.index.date
agg_n=agg.rename(columns={"close": "AGG"})
agg_n.head()

spy= spy.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
spy.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
spy.index = df.index.date
spy_n=spy.rename(columns={"close": "SPY"})
spy_n.head()



# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

# Calculate the daily returns for  AGG closing prices

In [ ]:
# Use the `pct_change` function to calculate daily returns of AAPL
daily_returns_agg = agg.pct_change()
daily_returns_agg.head()

# Calculate the daily returns for the SPY   closing prices.

In [ ]:
# Use the `pct_change` function to calculate daily returns of AAPL
daily_returns_spy = spy.pct_change()
daily_returns_spy.head()

# Calculate volatility  SPY  

In [ ]:
# Use the `std` function to calculate the standard deviation of daily returns for AAPL
std_dev_daily_return_spy = daily_returns_spy.std()['close']
std_dev_daily_return_spy

# Calculate volatility   AGG  

In [ ]:
# Use the `std` function to calculate the standard deviation of daily returns for AAPL
std_dev_daily_return_agg = daily_returns_agg.std()['close']
std_dev_daily_return_agg

# Find the last day's closing price for both stocks and save those as variables.

In [ ]:
# Set number of trading days and get last closing price of AAPL from DataFrame
num_trading_days = 252 *30
num_simulations = 500
spy_last_price =spy_n['SPY'][-1]
agg_last_price =agg_n['AGG'][-1]
spy_last_price 

# Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

In [ ]:
# Initialize the simulated prices list with the last closing price of AAPL
simulated_spy_prices = [spy_last_price]
avg_daily_return_spy = daily_returns_spy.mean()['close']

# Initialize empty DataFrame to hold simulated prices for each simulation
simulated_spyprice_df = pd.DataFrame()

# Run the simulation of projecting stock prices for the next trading year, `1000` times
for n in range(num_simulations):

    # Initialize the simulated prices list with the last closing price of AAPL
    simulated_spy_prices = [spy_last_price]
    
    # Simulate the returns for 252 days
    for i in range(num_trading_days):
        # Calculate the simulated price using the last price within the list
        simulated_price_spy = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_return_spy, std_dev_daily_return_spy))
        # Append the simulated price to the list
        simulated_spy_prices.append(simulated_price_spy)
    
    # Append a simulated prices of each simulation to DataFrame
    simulated_spyprice_df[f"Simulation {n+1}"] = pd.Series(simulated_spy_prices)

# Print head of DataFrame
simulated_spyprice_df.head()

In [ ]:
# Calculate the # Initialize the simulated prices list with the last closing price of AAPL
simulated_agg_prices = [agg_last_price]
avg_daily_return_agg = daily_returns_agg.mean()['close']

# Initialize empty DataFrame to hold simulated prices for each simulation
simulated_aggprice_df = pd.DataFrame()

# Run the simulation of projecting stock prices for the next trading year, `1000` times
for n in range(num_simulations):

    # Initialize the simulated prices list with the last closing price of AAPL
    simulated_agg_prices = [agg_last_price]
    
    # Simulate the returns for 252 days
    for i in range(num_trading_days):
        # Calculate the simulated price using the last price within the list
        simulated_price_agg = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg))
        # Append the simulated price to the list
        simulated_agg_prices.append(simulated_price_agg)
    
    # Append a simulated prices of each simulation to DataFrame
    simulated_aggprice_df[f"Simulation {n+1}"] = pd.Series(simulated_agg_prices)

# Print head of DataFrame
simulated_aggprice_df.head() 


In [8]:
 # Use the `plot` function to plot the trajectory of SPY stock based on a 252 trading day simulation
plot_title = f"{n+1} Simulations of SPY Stock Price Trajectory Over the Next 252 Trading Days"
simulated_spyprice_df.plot(legend=None, title=plot_title)

In [9]:
 # Use the `plot` function to plot the trajectory of AGG stock based on a 252 trading day simulation
plot_title = f"{n+1} Simulations of SPY Stock Price Trajectory Over the Next 252 Trading Days"
simulated_aggprice_df.plot(legend=None, title=plot_title)

In [14]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
# YOUR CODE HERE

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)
# YOUR CODE HERE

In [ ]:
# Display the 90% confidence interval for the ending returns
# YOUR CODE HERE

In [ ]:
# Visualize the distribution of the ending returns
# YOUR CODE HERE

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# YOUR CODE HERE

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# YOUR CODE HERE

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE